In [24]:
from torchvision.datasets import ImageFolder
from torchvision import transforms

from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

from torch.utils.data import Dataset
from PIL import Image
import os
import pandas as pd

### Loading Images

In [35]:
class CSVImageDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.labels_df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.labels_df.iloc[idx, 0])  
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = self.labels_df.iloc[idx, 1]  
        return image, label




In [36]:
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128,128))
])

csv_file_path = "Data/train.csv"
image_folder_path = "Data/train"

dataset_train = CSVImageDataset(
    csv_file=csv_file_path,
    root_dir=image_folder_path,
    transform=train_transforms,
)

print("Number of samples:", len(dataset_train))

Number of samples: 960


### Data Augmentation

In [48]:
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(45), 
    transforms.ToTensor(),
    transforms.Resize((128, 128)),
])

csv_file_path = "Data/train.csv"
image_folder_path = "Data/train"

dataset_train = CSVImageDataset(
    csv_file=csv_file_path,
    root_dir=image_folder_path,
    transform=train_transforms,
)

print("Number of samples:", len(dataset_train))

Number of samples: 960


### Linear layer for image? No
- slow training
- Overfitting
- Don't reognize spatial patterns

### Convolutional layer!!

In [45]:
class Net(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1), 
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2),
        )

        self.classifier = nn.Linear(65536, num_classes)
    
    def forward(self, x):  
        x = self.feature_extractor(x)
        x = x.view(x.size(0), -1)  
        x = self.classifier(x)
        return x

In [49]:

train_loader = DataLoader(dataset_train, shuffle=True)

num_classes = 7  
net = Net(num_classes=num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)


for epoch in range(3):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

c:\Users\juang\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1, Loss: 2.2377
Epoch 2, Loss: 2.1989
Epoch 3, Loss: 1.5874


In [44]:

train_loader.shape()

AttributeError: 'DataLoader' object has no attribute 'shape'